In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.5.0


In [2]:
!python -V

Python 3.11.9


In [3]:
import pickle
import pandas as pd
import polars as pl
import numpy as np

In [4]:
with open('../models/model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [5]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pl.scan_parquet(filename)

    df = df.with_columns(
        duration=(pl.col('tpep_dropoff_datetime') - pl.col('tpep_pickup_datetime')).dt.total_seconds() / 60
    ).filter(
        (pl.col('duration') >= 1) & (pl.col('duration') <= 60)
    ).with_columns(
        pl.col('PULocationID').fill_null(value=-1).cast(pl.Int16).cast(pl.Utf8).alias('PULocationID'),
        pl.col('DOLocationID').fill_null(value=-1).cast(pl.Int16).cast(pl.Utf8).alias('DOLocationID')
    )

    df = df.select(categorical+['duration'])

    return df.collect().to_pandas()


In [6]:
year = 2023
month = 3
taxi_type = 'yellow'

In [7]:
input_file = f"https://d37ci6vzurychx.cloudfront.net/trip-data/{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet"
output_file = f"../outputs/{taxi_type}_tripdata_{year:04d}-{month:02d}_prediction.parquet"

In [8]:
df = read_data(input_file)

In [9]:
print("Shape of processed dataframe: {df.shape}")

Shape of processed dataframe: {df.shape}


In [10]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [11]:
print(f"Mean of the prediction is: {np.mean(y_pred)}")
print(f"Standard deviation of the prediction is: {np.std(y_pred)}")

Mean of the prediction is: 14.203865642696083
Standard deviation of the prediction is: 6.247488852238703


In [12]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [13]:
df_result = pd.DataFrame()

In [14]:
df_result['ride_id'] = df['ride_id']
df_result['predicted_duration'] = y_pred

In [15]:
output_file = "../outputs/yellow_tripdata_2023-03_prediction.parquet"

In [16]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [17]:
print(f"Result DataFrame contains two columns: {[col for col in df_result.columns]}")

Result DataFrame contains two columns: ['ride_id', 'predicted_duration']


In [18]:
df = pd.DataFrame()